In [ ]:
!pip install psycopg2 # library for postrgres connector
from sqlalchemy import create_engine
import pandas as pd

In [29]:
# connect to engine. This assumes a DB of saddl, and root user without a pw
engine = create_engine('postgresql://:@localhost:5432/saddl')

In [38]:
with engine.connect() as con:
    with open('../db-export/0-0-1-prerelease/schema.sql', mode='r') as f:
        cmds = []
        for line in f.readlines():
            if 'CREATE TABLE' in line:
                line = line.strip().replace('BIGINT', 'INT').replace('FLOAT', 'REAL').replace('DOUBLE', 'REAL')
                print(line)
                result = con.execute(line)
                print(result.all())

['CREATE TABLE work_predictions(target INT, candidate INT, swsm REAL, swde REAL, wp_dv REAL, partof REAL, contains REAL, "overlaps" REAL, author REAL, simdiff REAL, grsim REAL, randdiff REAL, count INTEGER);',
 'CREATE TABLE clean_predictions(target VARCHAR, candidate VARCHAR, swsm REAL, swde REAL, wp_dv REAL, partof REAL, contains REAL, "overlaps" REAL, author REAL, simdiff REAL, grsim REAL, randdiff REAL, relatedness REAL, count INTEGER);',
 'CREATE TABLE meta(htid VARCHAR, "access" VARCHAR, rights VARCHAR, ht_bib_key VARCHAR, description VARCHAR, source VARCHAR, source_bib_num VARCHAR, oclc_num VARCHAR, isbn VARCHAR, issn VARCHAR, lccn VARCHAR, title VARCHAR, imprint VARCHAR, rights_reason_code VARCHAR, rights_timestamp VARCHAR, us_gov_doc_flag INT, rights_date_used REAL, pub_place VARCHAR, lang VARCHAR, bib_fmt VARCHAR, collection_code VARCHAR, content_provider_code VARCHAR, responsible_entity_code VARCHAR, digitization_agent_code VARCHAR, access_profile_code VARCHAR, author VARCHA

In [27]:
import os
with engine.connect() as con:
    with open('../db-export/0-0-1-prerelease/load.sql', mode='r') as f:
        copy_cmds = []
        for copy_cmd in f.readlines():
            if 'COPY' not in line:
                continue
            a, b = copy_cmd.split(' FROM \'')
            _, table_name = a.split()
            parquet_location = b.split("' (FORMAT")[0].strip("'")
            csv_tmp_path = parquet_location.replace('parquet', 'csv.tmp')
            pd.read_parquet(parquet_location).to_csv(csv_tmp_path, index=False, header=True)
            pq_cmd = f"COPY {table_name} FROM '{csv_tmp_path}' DELIMITER ',' CSV HEADER;"
            print(pq_cmd)
            con.execute(pq_cmd)
            os.remove(csv_tmp_path)

COPY work_stats FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/6_work_stats.csv.tmp' DELIMITER ',' CSV HEADER;
COPY work_predictions FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/0_work_predictions.csv.tmp' DELIMITER ',' CSV HEADER;
COPY manifestation_predictions FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/4_manifestation_predictions.csv.tmp' DELIMITER ',' CSV HEADER;
COPY meta FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/2_meta.csv.tmp' DELIMITER ',' CSV HEADER;
COPY clean_predictions FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/1_clean_predictions.csv.tmp' DELIMITER ',' CSV HEADER;
COPY clusters FROM '/Users/organisciak/Documents/projects/saddl-website/db-export/0-0-1-prerelease/5_clusters.csv.tmp' DELIMITER ',' CSV HEADER;


## ADD PRIMARY KEY

`ALTER TABLE` locks the table, so make sure there's nothing else blocking your query.

https://stackoverflow.com/questions/26489244/how-to-detect-query-which-holds-the-lock-in-postgres

https://stackoverflow.com/questions/35319597/how-to-stop-kill-a-query-in-postgresql

In [ ]:
with engine.connect() as con:
    con.execute('ALTER TABLE work_stats ADD PRIMARY KEY (work_id);')
    con.execute('ALTER TABLE meta ADD PRIMARY KEY (htid);')
    con.execute('ALTER TABLE clusters ADD PRIMARY KEY (htid);')

## CREATE INDEX

This makes a huge difference.

In [55]:
with engine.connect() as con:
    con.execute('CREATE INDEX target_idx ON clean_predictions (target);')
    con.execute('CREATE INDEX candidate_idx ON clean_predictions (candidate);')

    con.execute('CREATE INDEX target_work_idx ON work_predictions (target);')
    con.execute('CREATE INDEX candidate_work_idx ON work_predictions (candidate);')

    con.execute('CREATE INDEX target_man_idx ON manifestation_predictions (target);')
    con.execute('CREATE INDEX candidate_man_idx ON manifestation_predictions (candidate);')

    con.execute('CREATE INDEX work_cluster_idx ON clusters (work_id);')
    con.execute('CREATE INDEX man_cluster_idx ON clusters (man_id);')

## Test

In [48]:
with engine.connect() as con:
    result = con.execute('SELECT * FROM work_stats LIMIT 1')
result.all()

[(55553, 5, 0, 5, 0.0, 1.0)]

In [49]:
pd.read_sql('SELECT * FROM work_stats LIMIT 1', engine)

,work_id,label_count,gov_count,serial_count,gov_prop,serial_prop
0,55553,5,0,5,0.0,1.0
